In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader as web
import math
from keras.models import Sequential
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

In [39]:
! jupyter nbextension enable --py --sys-prefix widgetsnbextension
! jupyter nbextension install --py --symlink --sys-prefix ipympl

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Installing c:\users\voops\appdata\local\programs\python\python38\lib\site-packages\ipympl\nbextension -> jupyter-matplotlib
- Validating: ok

    To initialize this nbextension in the browser every time the notebook (or other app) loads:
    
          jupyter nbextension enable ipympl --py --sys-prefix
    


In [40]:
%matplotlib widget

# Data gathering

In [41]:
day = int(input('Какой Сегодня день?', ))
month = int(input('Какого месяца?', ))
year = int(input('Какого года?', ))
today = f'{day}-{month}-{year}'
print('Сегодня ', today)

Какой Сегодня день?26
Какого месяца?06
Какого года?2021
Сегодня  26-6-2021


In [42]:
data = web.DataReader('CL=F', data_source='yahoo', start='1980-01-01', end=today)

In [43]:
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-08-23,32.799999,31.950001,31.950001,32.049999,79385.0,32.049999
2000-08-24,32.240002,31.400000,31.900000,31.629999,72978.0,31.629999
2000-08-25,32.099998,31.320000,31.700001,32.049999,44601.0,32.049999
2000-08-28,32.919998,31.860001,32.040001,32.869999,46770.0,32.869999
2000-08-29,33.029999,32.560001,32.820000,32.720001,49131.0,32.720001
...,...,...,...,...,...,...
2021-06-21,73.959999,71.150002,71.519997,73.660004,123662.0,73.660004
2021-06-22,73.949997,72.940002,73.410004,73.059998,498555.0,73.059998
2021-06-23,74.250000,72.820000,72.910004,73.080002,429207.0,73.080002


In [44]:
data.shape

(5201, 6)

# Preprocessing

In [45]:
data.drop(data.loc[data['Close']<0].index, inplace=True)

In [46]:
plt.figure()
plt.title('Price History')
plt.plot(data['Close'])
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid('True')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
df = data.filter(['Close'])
df_val = df.values
train_len = math.ceil(len(df)*0.8)

In [48]:
df.describe()

,Close
count,5200.000000
mean,61.807898
std,25.593085
min,10.010000
25%,41.914999
50%,58.645000
75%,81.250000
max,145.179993


In [49]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_val)

In [50]:
scaled_data

array([[0.16305393],
       [0.15994673],
       [0.16305393],
       ...,
       [0.46659766],
       [0.46822524],
       [0.47377381]])

In [51]:
train_data = scaled_data[:train_len, :]

X_train=[]
y_train=[]

for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])


In [52]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [53]:
X_train = np.expand_dims(X_train, 1)

# Model

In [54]:
model = Sequential(        [
        layers.Dense(16, activation='relu', input_shape=(X_train.shape[1:])),
        layers.LSTM(64, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(64, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(64),
        layers.Dropout(0.4),
        layers.Flatten(),
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='elu'),
        layers.Dense(1)
        ])

In [55]:
model.compile(optimizer='adam', loss='mse')

# Training

In [56]:
model.fit(X_train, y_train, batch_size=128, epochs=100)

Epoch 1/100
33/33 [==============================] - 5s 3ms/step - loss: 0.1400
Epoch 2/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0179
Epoch 3/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0090
Epoch 4/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0060
Epoch 5/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0053
Epoch 6/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0046
Epoch 7/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0041
Epoch 8/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0033
Epoch 9/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0028
Epoch 10/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0023
Epoch 11/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0022
Epoch 12/100
33/33 [==============================] - 0s 3ms/step - loss: 0.0020
Epoch 13/100
33/33 [=================

33/33 [==============================] - 0s 3ms/step - loss: 6.9855e-04
Epoch 100/100
33/33 [==============================] - 0s 3ms/step - loss: 6.3691e-04


In [57]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_11 (ModuleWra (None, 1, 16)             976       
_________________________________________________________________
module_wrapper_12 (ModuleWra (None, 1, 64)             20736     
_________________________________________________________________
module_wrapper_13 (ModuleWra (None, 1, 64)             0         
_________________________________________________________________
module_wrapper_14 (ModuleWra (None, 1, 64)             33024     
_________________________________________________________________
module_wrapper_15 (ModuleWra (None, 1, 64)             0         
_________________________________________________________________
module_wrapper_16 (ModuleWra (None, 64)                33024     
_________________________________________________________________
module_wrapper_17 (ModuleWra (None, 64)               

In [58]:
test_data = scaled_data[train_len - 60: , :]

X_test = []
y_test = df_val[train_len:, :]
for i in range(60, len(test_data)):
    X_test.append(test_data[i-60:i, 0])
    

In [59]:
X_test = np.array(X_test)

In [60]:
X_test = np.expand_dims(X_test, 1)

In [61]:
X_test.shape

(1040, 1, 60)

In [62]:
pred = model.predict(X_test)
pred = scaler.inverse_transform(pred)

# Metrics

In [63]:
train = data[:train_len]
valid = data[train_len:]
valid['Predictions'] = pred
valid
fig = plt.figure()
plt.xlabel('Дата', fontsize = 18)
plt.ylabel('USD', fontsize = 18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.grid(True)

<ipython-input-63-c5a5aa341c12>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = pred


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [65]:
A = valid['Close']
F = valid['Predictions']
print('sMAPE = ',smape(A, F))

sMAPE =  3.2663803854483695


In [66]:
pred.shape

(1040, 1)

In [67]:
valid[['Close', 'Predictions']]

,Close,Predictions
Date,,
2017-04-25,49.560001,50.809364
2017-04-26,49.619999,50.261143
2017-04-27,48.970001,50.142143
2017-04-28,49.330002,49.827892
2017-05-01,48.840000,49.672371
...,...,...
2021-06-21,73.660004,72.300423
2021-06-22,73.059998,73.074371
2021-06-23,73.080002,73.056976


# Future forecast 

In [68]:
oil_price = web.DataReader('CL=F', data_source='yahoo', start='1980-01-01', end=today)
dafr = oil_price.filter(['Close'])
dafr_vals = dafr[-60:].values
dafr_scaled = scaler.transform(dafr_vals)
xtest = []
xtest.append(dafr_scaled)
xtest = np.array(xtest)
xtest = np.reshape(xtest, (1, 1, 60))

abobka = []

for i in range(30):
    prediction = model.predict(xtest)
    temp = xtest[0][0]
    temp_arr = np.array(temp[1:])
    temp_arr = np.append(temp_arr, prediction)
    xtest = np.reshape(temp_arr, (1, 1, 60))
    prediction = scaler.inverse_transform(prediction)
    abobka.append(prediction)

In [69]:
labobka=[]
for i in abobka:
    labobka.append(i[0][0])

In [70]:
dti = pd.date_range(f'{day+1}-{month}-{year}', periods=30, freq="D")

In [71]:
next_30d = {'Date':dti,
           'Predicted price USD':labobka}
nextdf = pd.DataFrame(next_30d, columns = ['Date', 'Predicted price USD'])

In [72]:
fig = plt.figure()
plt.title('Прогноз на следующие 30 дней', fontsize = 18)
plt.xlabel('Дата', fontsize = 18)
plt.ylabel('USD', fontsize = 18)
plt.plot( nextdf['Date'], nextdf['Predicted price USD'])
plt.grid(True)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
nextdf

,Date,Predicted price USD
0,2021-06-27,73.944183
1,2021-06-28,73.874733
2,2021-06-29,74.760590
3,2021-06-30,75.150169
4,2021-07-01,75.145325
5,2021-07-02,75.231560
6,2021-07-03,75.775696
7,2021-07-04,76.306679
8,2021-07-05,76.557808
9,2021-07-06,76.790489
